In [29]:
import sqlite3
import matplotlib
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import utility as util
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix, f1_score


In [30]:
neo_lab_first_day = pd.read_csv('neo_lab_first_day.csv')

In [31]:
neo_lab_first_day.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'ANIONGAP_min', 'ANIONGAP_max',
       'ALBUMIN_min', 'ALBUMIN_max', 'BANDS_min', 'BANDS_max',
       'BICARBONATE_min', 'BICARBONATE_max', 'BILIRUBIN_min', 'BILIRUBIN_max',
       'CREATININE_min', 'CREATININE_max', 'CHLORIDE_min', 'CHLORIDE_max',
       'GLUCOSE_min', 'GLUCOSE_max', 'HEMATOCRIT_min', 'HEMATOCRIT_max',
       'HEMOGLOBIN_min', 'HEMOGLOBIN_max', 'LACTATE_min', 'LACTATE_max',
       'PLATELET_min', 'PLATELET_max', 'POTASSIUM_min', 'POTASSIUM_max',
       'PTT_min', 'PTT_max', 'INR_min', 'INR_max', 'PT_min', 'PT_max',
       'SODIUM_min', 'SODIUM_max', 'BUN_min', 'BUN_max', 'WBC_min', 'WBC_max',
       'HADM_ID', 'Unnamed: 0', 'SUBJECT_ID', '30readflag'],
      dtype='object')

In [32]:
neo_lab_first_day['PLATELET_min'].value_counts()

264.0    53
290.0    49
295.0    46
253.0    42
318.0    42
         ..
545.0     1
528.0     1
127.0     1
507.0     1
518.0     1
Name: PLATELET_min, Length: 513, dtype: int64

In [33]:
neo_bloog_gas_all_time_dropna = neo_lab_first_day.dropna(axis= 1, how = 'all')
pco2_score = neo_bloog_gas_all_time_dropna['POTASSIUM_max']
pco2_score.value_counts()

5.3     117
4.9     117
5.2     112
5.8     110
5.5     108
       ... 
9.8       2
2.5       1
3.2       1
12.5      1
9.7       1
Name: POTASSIUM_max, Length: 70, dtype: int64

In [34]:
pco2_score2 = pco2_score
pco2_score[(pco2_score2.values>9)]  = 5
pco2_score[(pco2_score2<7)] = 1
pco2_score[(pco2_score2>=7) & (pco2_score2<=9)] = 3
#pco2_score[np.isnan(pco2_score2)] = 1

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in greater
  
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: h

In [35]:
pco2_score = pco2_score.fillna(1)

In [36]:
neo_bloog_gas_all_time_dropna['potassium_max_score'] = pco2_score

In [40]:
neo_basic_label_df_feature_SNAP = pd.read_csv('neo_los_basic_df.csv')
sub = pd.DataFrame(neo_bloog_gas_all_time_dropna[['potassium_max_score','HADM_ID']])
neo_basic_label_df_feature_SNAP = util.merge(neo_basic_label_df_feature_SNAP,sub, 'inner','HADM_ID')
neo_basic_label_df_feature_SNAP.to_csv('potassium_max_score.csv',index = False)
neo_bloog_gas_all_time_dropna =neo_basic_label_df_feature_SNAP
neo_bloog_gas_all_time_dropna

,Unnamed: 0,sum_los,HADM_ID,los_bin,potassium_max_score
0,0,0.3638,100023,0,1.0
1,1,3.0070,100025,0,1.0
2,2,30.5118,100029,1,1.0
3,3,65.7146,100044,1,1.0
4,4,3.9160,100055,0,1.0
...,...,...,...,...,...
7969,7987,48.2892,199913,1,1.0
7970,7988,19.3016,199917,0,1.0
7971,7989,103.9422,199918,1,1.0
7972,7990,0.1066,199954,0,1.0


In [41]:
cm00 = np.sum((neo_bloog_gas_all_time_dropna['los_bin']==0)& (neo_bloog_gas_all_time_dropna['potassium_max_score']<=1))
cm00
cm01 = np.sum((neo_bloog_gas_all_time_dropna['los_bin']==0)&(neo_bloog_gas_all_time_dropna['potassium_max_score']>1))
cm00
cm10 = np.sum((neo_bloog_gas_all_time_dropna['los_bin']==1)&(neo_bloog_gas_all_time_dropna['potassium_max_score']<=1))
cm10
cm11 = np.sum((neo_bloog_gas_all_time_dropna['los_bin']==1)&(neo_bloog_gas_all_time_dropna['potassium_max_score']>1))
cm11

cm = np.array([[cm00,cm01],[cm10,cm11]])

In [42]:
neo_basic_label_df_feature_SNAP

,Unnamed: 0,sum_los,HADM_ID,los_bin,potassium_max_score
0,0,0.3638,100023,0,1.0
1,1,3.0070,100025,0,1.0
2,2,30.5118,100029,1,1.0
3,3,65.7146,100044,1,1.0
4,4,3.9160,100055,0,1.0
...,...,...,...,...,...
7969,7987,48.2892,199913,1,1.0
7970,7988,19.3016,199917,0,1.0
7971,7989,103.9422,199918,1,1.0
7972,7990,0.1066,199954,0,1.0


In [43]:
print(cm)
print(cm01/(cm00+cm01))
print(cm11/(cm10+cm11))


[[5942   62]
 [1758  212]]
0.010326449033977348
0.10761421319796954


In [44]:
util.chi2(cm)

6004.0
chi 423.11487969631577
pval 5.122261588011322e-94


(423.11487969631577, 5.122261588011322e-94)